In [88]:
f = open("./VIDEO.dump","rb")
o = open("./VIDEO.264","wb")
udp = f.read(0x580)
while udp:
    udp = f.read(0x580)
    o.write(udp[0x28:0x580])
    udp = f.read(0x580)
f.close()
o.close()

In [ ]:
# https://github.com/LizardByte/Sunshine/blob/25ed2d5b4a5bde402bc573b5dd1c8479757a9735/src/platform/linux/input/inputtino_keyboard.cpp
key ={"DD":'}', "DB":'{', "A0":'^'}

f = open("./sunshine.log")
r = f.readline()
act = ""
ans = ""
while r:
    if r[-26:-1] == "--begin keyboard packet--":
        r1 = f.readline()
        r2 = f.readline()
        if r1 == "keyAction [00000003]\n":
            act = act + "{ "
        elif r1 == "keyAction [00000004]\n":
            act = act + " }"
        else:
            act = act + "$"

        t = int(r2[11:13],16)
        if t >=32 and t<=126:
            ans = ans+chr(t)+"$"
        elif r2[11:13] in key.keys():
            ans = ans+key[r2[11:13]]+"$"
        else:
            ans = ans+r2[11:13]
    r = f.readline()
    
print(act)
print(ans)
f.close()

In [10]:
# Hint

# <uniqueid>A7EDB273-EABA-3207-5814-8721BC5EDD30</uniqueid> ... KEY?

# <uniqueid> 0123456789ABCDEF ... VI?

# [2024:09:30:17:14:25]: Debug: rikeyid -- 1485042510
# [2024:09:30:17:14:25]: Debug: rikey -- F3CB8CFA676D563BBEBFC80D3943F10A

import struct
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad

def decrypt_audio_pkt(seq, payload):

    iv = struct.pack('>i', int('1485042510')+seq) + b'\x00'*12 # https://github.com/LizardByte/Sunshine/blob/190ea41b2ea04ff1ddfbe44ea4459424a87c7d39/src/stream.cpp#L1516
    cipher = AES.new(b'\xF3\xCB\x8C\xFA\x67\x6D\x56\x3B\xBE\xBF\xC8\x0D\x39\x43\xF1\x0A', AES.MODE_CBC, iv)
    # https://github.com/moonlight-stream/moonlight-common-c/blob/master/src/AudioStream.c#L192
    
    return unpad(cipher.decrypt(payload), 16)

f = open("./OPUS.pcap","rb")
o = open("./DECRYPT.pcap","wb")

h = f.read(0x18)
o.write(h)

bs = f.read(0xC6)
while bs:
    tol2 = int.from_bytes(bs[0x20:0x22],byteorder='big') # +20
    tol1 = int.from_bytes(bs[0x36:0x38],byteorder='big') # +20
    seq = int.from_bytes(bs[0x3c:0x3e],byteorder='big')
    encrypt = bs[0x46:0xC6]
    decrypt = decrypt_audio_pkt(seq, encrypt) 
    # h += (bs[0x00:0x08]+b"\xae\x00\x00\x00\xae\x00\x00\x00")
    o.write(bs[0x00:0x08]+b"\xae\x00\x00\x00\xae\x00\x00\x00")
    # h += (bs[0x10:0x20]+b"\x00\xa0"+bs[0x22:0x32])
    o.write(bs[0x10:0x20]+b"\x00\xa0"+bs[0x22:0x32])
    # h += (bs[0x32:0x36]+b"\x00\x8c"+bs[0x38:0x3a]+bs[0x3a:0x46])
    o.write(bs[0x32:0x36]+b"\x00\x8c"+bs[0x38:0x3a]+bs[0x3a:0x46])
    # h += decrypt
    o.write(decrypt)
    bs = f.read(0xC6)

f.close()
o.close()